In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#this dataset is giving us the conditions required to grow a particular crop
#no info on where these conditions are found
# Crop_Production_Statistics does not have rainfall, thus using crop_yield

soil=pd.read_csv('/kaggle/input/crop-recommendation/Crop_recommendation.csv')
crops=pd.read_csv('/kaggle/input/crop-yield/crop_yield.csv')
# crops=pd.read_csv('/kaggle/input/india-crop-yield/Crop_Production_Statistics.csv')

In [ ]:
soil.head()

In [ ]:
crops.head()

In [ ]:
soil['rainfall']=np.ceil(soil['rainfall'])

In [ ]:
crops['State'].nunique()

In [ ]:
crops.State.unique()

In [ ]:
crops.Crop.unique()

In [ ]:
crops['mean_rainfall']=np.ceil(crops['Annual_Rainfall']/12)
crops.head()

In [ ]:
df = pd.DataFrame(soil)

# Filter the dataframe
soil_filtered_df = df[(df['label']=='maize')]

# Print the filtered dataframe
print(soil_filtered_df)

In [ ]:
soil.label.unique()

In [ ]:
df = pd.DataFrame(crops)

# Filter the dataframe
crop_filtered_df = df[(df['Crop']=='Maize')]

# Print the filtered dataframe
print(crop_filtered_df)

In [ ]:
merged_df = pd.merge(soil_filtered_df, crop_filtered_df, left_on=['rainfall'], right_on=['mean_rainfall'])
print(merged_df)

In [ ]:
merged_df.Crop_Year.unique()

In [ ]:
merged_df=merged_df.drop(columns= ['label', 'Crop','mean_rainfall'])

In [ ]:
merged_df.to_csv('merged_df.csv',index=False)
merged_df.head()

In [ ]:
merged_df.info()

In [ ]:
merged_df.sample(5)

In [ ]:
pip install pandas seaborn matplotlib

# LABEL ENCODING NON-NUMERIC COLUMNS

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming merged_df is your DataFrame
# Identify non-numeric columns
non_numeric_columns = merged_df.select_dtypes(include=['object']).columns

# Initialize the label encoder
label_encoders = {}
for column in non_numeric_columns:
    label_encoders[column] = LabelEncoder()
    merged_df[column] = label_encoders[column].fit_transform(merged_df[column])

# Display the first few rows of the updated DataFrame
print(merged_df.head(15))


In [ ]:
merged_df_1=merged_df.drop(columns=['Production'])
merged_df_1.to_csv('merged_df_1.csv',index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming merged_df is your DataFrame
# Identify non-numeric columns
non_numeric_columns = merged_df_1.select_dtypes(include=['object']).columns

# Initialize the label encoder
label_encoders = {}
for column in non_numeric_columns:
    label_encoders[column] = LabelEncoder()
    merged_df_1[column] = label_encoders[column].fit_transform(merged_df_1[column])

# Display the first few rows of the updated DataFrame
print(merged_df_1.head(15))

# GENERATING HEATMAP

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(42,42))
sns.heatmap(merged_df.corr(), annot=True)
plt.savefig('/kaggle/working/correlation_heatmap.png')
plt.show()

In [ ]:
plt.figure(figsize=(42,42))
sns.heatmap(merged_df_1.corr(), annot=True)
plt.savefig('/kaggle/working/correlation_heatmap_wo_prod.png')
plt.show()

# RANDOM FOREST

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# Display the first few rows of the dataset
print(data.head())

# Separate features and target variable
X = data.drop(columns=['Yield'])
y = data['Yield']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year','Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')


# PROPHET

In [ ]:
# pip install prophet

In [ ]:
# import pandas as pd
# from prophet import Prophet
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import numpy as np

# # Load the dataset
# data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# # Drop the 'Crop' column since it has only one type of crop
# # data = data.drop(columns=['Crop'])

# # Strip whitespace from 'Season' values
# data['Season'] = data['Season'].str.strip()

# # Convert 'Crop_Year' and 'Season' into a datetime format
# # Map seasons to months (assuming a mapping)
# season_to_month = {
#     'Spring': '03-01',
#     'Summer': '06-01',
#     'Autumn': '09-01',
#     'Winter': '12-01',
#     'Kharif': '07-01',  # Example for Kharif season
#     'Rabi': '01-01'    # Example for Rabi season
# }

# # Handle missing or unexpected seasons
# def get_season_month(season):
#     return season_to_month.get(season, '01-01')

# data['Date'] = data.apply(lambda row: f"{row['Crop_Year']}-{get_season_month(row['Season'])}", axis=1)
# data['Date'] = pd.to_datetime(data['Date'])

# # Prepare the dataframe for Prophet
# df = data.rename(columns={'Date': 'ds', 'Yield': 'y'})
# df = df[['ds', 'y', 'Annual_Rainfall', 'Fertilizer', 'Pesticide', 'rainfall']]

# # Initialize the Prophet model
# model = Prophet()

# # Add additional regressors
# model.add_regressor('Annual_Rainfall')
# model.add_regressor('Fertilizer')
# model.add_regressor('Pesticide')
# # model.add_regressor('Production')
# model.add_regressor('rainfall')

# # Fit the model
# model.fit(df)

# # Make future dataframe for predictions
# future = model.make_future_dataframe(periods=12, freq='M')
# # Fill the future dataframe with the mean of the regressors for now
# future['Annual_Rainfall'] = df['Annual_Rainfall'].mean()
# future['Fertilizer'] = df['Fertilizer'].mean()
# future['Pesticide'] = df['Pesticide'].mean()
# # future['Production'] = df['Production'].mean()
# future['rainfall'] = df['rainfall'].mean()

# # Make predictions
# forecast = model.predict(future)

# # Evaluate the model on the historical data
# df_forecast = forecast[['ds', 'yhat']].merge(df, on='ds')
# mae = mean_absolute_error(df_forecast['y'], df_forecast['yhat'])
# mse = mean_squared_error(df_forecast['y'], df_forecast['yhat'])
# rmse = np.sqrt(mse)
# r2 = r2_score(df_forecast['y'], df_forecast['yhat'])

# print(f'MAE: {mae}')
# print(f'MSE: {mse}')
# print(f'RMSE: {rmse}')
# print(f'R2: {r2}')

# # Plot the forecast
# model.plot(forecast)


# Back Propagation Neural Network (BPNN)

In [ ]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# import numpy as np

# # Load the dataset
# data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# # Drop the 'Crop' column since it has only one type of crop
# # data = data.drop(columns=['Crop'])

# # Separate features and target variable
# X = data.drop(columns=['Yield'])
# y = data['Yield']

# # Identify categorical and numerical columns
# categorical_cols = ['Crop_Year', 'Season', 'State']
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# # Preprocessing for numerical data: impute missing values and scale features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# # Preprocessing for categorical data: impute missing values and one-hot encode
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Combine preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Apply preprocessing to the features
# X_preprocessed = preprocessor.fit_transform(X)

# # Convert sparse matrix to dense
# X_preprocessed = X_preprocessed.toarray()

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Build the Neural Network
# model = Sequential()
# model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1))

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print(f'MAE: {mae}')
# print(f'MSE: {mse}')
# print(f'RMSE: {rmse}')
# print(f'R2: {r2}')


# Support Vector Regression (SVR)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Load the dataset
data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# Drop the 'Crop' column since it has only one type of crop
# data = data.drop(columns=['Crop'])

# Separate features and target variable
X = data.drop(columns=['Yield'])
y = data['Yield']

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year', 'Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the features
X_preprocessed = preprocessor.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Train the SVR model
svr = SVR(kernel='rbf')
svr.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svr.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R2: {r2}')


# Gaussian Process Regression (GPR)

In [ ]:
# # gpr on yield

# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# # Load the dataset
# data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# # Drop the 'Crop' column since it has only one type of crop
# # data = data.drop(columns=['Crop'])

# # Separate features and target variable
# X = data.drop(columns=['Yield'])
# y = data['Yield']

# # Identify categorical and numerical columns
# categorical_cols = ['Crop_Year', 'Season', 'State']
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# # Preprocessing for numerical data: impute missing values and scale features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# # Preprocessing for categorical data: impute missing values and one-hot encode
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Combine preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Apply preprocessing to the features
# X_preprocessed = preprocessor.fit_transform(X)

# # Convert the sparse matrix to a dense format
# X_preprocessed = X_preprocessed.toarray()

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Define the kernel for GPR
# kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))

# # Create and fit the GPR model
# gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
# gpr.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred, y_std = gpr.predict(X_test, return_std=True)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print(f'MAE: {mae}')
# print(f'MSE: {mse}')
# print(f'RMSE: {rmse}')
# print(f'R2: {r2}')


In [ ]:
# # gpr on production

# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from sklearn.gaussian_process import GaussianProcessRegressor
# from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

# # Load the dataset
# data = pd.read_csv('/kaggle/working/merged_df.csv')

# # Drop the 'Crop' column since it has only one type of crop
# # data = data.drop(columns=['Crop'])

# # Separate features and target variable
# X = data.drop(columns=['Production'])
# y = data['Production']

# # Identify categorical and numerical columns
# categorical_cols = ['Crop_Year', 'Season', 'State']
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# # Preprocessing for numerical data: impute missing values and scale features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# # Preprocessing for categorical data: impute missing values and one-hot encode
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Combine preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Apply preprocessing to the features
# X_preprocessed = preprocessor.fit_transform(X)

# # Convert the sparse matrix to a dense format
# X_preprocessed = X_preprocessed.toarray()

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Define the kernel for GPR
# kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))

# # Create and fit the GPR model
# gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
# gpr.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred, y_std = gpr.predict(X_test, return_std=True)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print(f'MAE: {mae}')
# print(f'MSE: {mse}')
# print(f'RMSE: {rmse}')
# print(f'R2: {r2}')


# XGBoost

In [ ]:
pip install xgboost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

# Load the dataset
data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# Separate features and target variable
X = data.drop(columns=['Yield'])
y = data['Yield']

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year', 'Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the features
X_preprocessed = preprocessor.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Initialize the XGBoost Regressor
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')


# LGBM

In [ ]:
# pip install lightgbm

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import lightgbm as lgb

# # Load the dataset
# data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# # Separate features and target variable
# X = data.drop(columns=['Yield'])
# y = data['Yield']

# # Identify categorical and numerical columns
# categorical_cols = ['Crop_Year', 'Season', 'State']
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# # Preprocessing for numerical data: impute missing values and scale features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# # Preprocessing for categorical data: impute missing values and one-hot encode
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Combine preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Apply preprocessing to the features
# X_preprocessed = preprocessor.fit_transform(X)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Initialize the LightGBM Regressor
# lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# # Train the model
# lgb_model.fit(X_train, y_train)

# # Make predictions on the test set
# y_pred = lgb_model.predict(X_test)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Absolute Error: {mae}')
# print(f'Mean Squared Error: {mse}')
# print(f'Root Mean Squared Error: {rmse}')
# print(f'R2 Score: {r2}')

# CATBoost

In [ ]:
pip install catboost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor

# Load the dataset
data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# Separate features and target variable
X = data.drop(columns=['Yield'])
y = data['Yield']

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year', 'Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the features
X_preprocessed = preprocessor.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Initialize the CatBoost Regressor
catboost_model = CatBoostRegressor(iterations=100, learning_rate=0.1, random_state=42, verbose=0)

# Train the model
catboost_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = catboost_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

# RNN

In [ ]:
# pip install tensorflow

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM, Dropout

# # Load the dataset
# data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# # Separate features and target variable
# X = data.drop(columns=['Yield'])
# y = data['Yield']

# # Identify categorical and numerical columns
# categorical_cols = ['Crop_Year', 'Season', 'State']
# numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# # Preprocessing for numerical data: impute missing values and scale features
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='mean')),
#     ('scaler', StandardScaler())
# ])

# # Preprocessing for categorical data: impute missing values and one-hot encode
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Combine preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, numerical_cols),
#         ('cat', categorical_transformer, categorical_cols)
#     ])

# # Apply preprocessing to the features
# X_preprocessed = preprocessor.fit_transform(X)

# # Convert the sparse matrix to a dense one
# X_preprocessed = X_preprocessed.toarray()

# # Reshape the data to fit the LSTM input format (samples, time steps, features)
# # Here we assume each sample is one time step
# X_preprocessed = X_preprocessed.reshape((X_preprocessed.shape[0], 1, X_preprocessed.shape[1]))

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# # Define the RNN model
# model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
# model.add(Dropout(0.2))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# # Make predictions on the test set
# y_pred = model.predict(X_test)

# # Evaluate the model
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = np.sqrt(mse)
# r2 = r2_score(y_test, y_pred)

# print(f'Mean Absolute Error: {mae}')
# print(f'Mean Squared Error: {mse}')
# print(f'Root Mean Squared Error: {rmse}')
# print(f'R2 Score: {r2}')

Next things to try:
1. drop yield and get results with production as output (not needed)
2. drop production and get results with yield as output (done)
3. apply random forest on a set of features to analyse most imp combo of features

In [ ]:
# cross validation on random forest 

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
data = pd.read_csv('/kaggle/working/merged_df_1.csv')

# Drop the 'Crop' column since it has only one type of crop
# data = data.drop(columns=['Crop'])

# Separate features and target variable
X = data.drop(columns=['Yield'])
y = data['Yield']

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year', 'Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the features
X_preprocessed = preprocessor.fit_transform(X)

# Define the cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Define scoring metrics
scoring = {
    'MAE': make_scorer(mean_absolute_error),
    'MSE': make_scorer(mean_squared_error),
    'R2': make_scorer(r2_score)
}

# Perform cross-validation and get the scores
cv_results = cross_validate(rf_model, X_preprocessed, y, cv=kf, scoring=scoring, return_train_score=True)

# Display cross-validation results
print("Cross-Validation Results:")
print(f"Mean Absolute Error: {np.mean(cv_results['test_MAE'])}")
print(f"Mean Squared Error: {np.mean(cv_results['test_MSE'])}")
print(f"R2 Score: {np.mean(cv_results['test_R2'])}")


In [ ]:
merged_df_1.head()

In [ ]:
merged_df_1.info()

# Comparing different attributes with RF

In [ ]:
# total 14 input attributes
# ignoring the categorical features Crop_Year, Season, State
# now have 11 options
# making random attribute sets from these 11 to find most imp attributes

set_output=merged_df_1['Yield']

In [ ]:
attr_set_1=['Crop_Year','Season', 'State','temperature','ph','rainfall','Fertilizer','Pesticide']
set_1=merged_df_1[attr_set_1].copy()

# Separate features and target variable
X = set_1
y = set_output

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year','Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

In [ ]:
attr_set_2=['Crop_Year','Season', 'State','N','P','K','humidity']
set_2=merged_df_1[attr_set_2].copy()
# Separate features and target variable
X = set_2
y = set_output

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year','Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

In [ ]:
attr_set_3=['Crop_Year','Season', 'State', 'temperature', 'ph', 'Fertilizer', 'N','P','K']
set_3=merged_df_1[attr_set_3].copy()

# Separate features and target variable
X = set_3
y = set_output

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year','Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

In [ ]:
attr_set_4=['Crop_Year','Season', 'State', 'temperature', 'ph', 'N','P','K']
set_4=merged_df_1[attr_set_4].copy()

# Separate features and target variable
X = set_4
y = set_output

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = ['Crop_Year','Season', 'State']
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.difference(categorical_cols)

# Preprocessing for numerical data: impute missing values and scale features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values and one-hot encode
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Apply preprocessing to the training and testing data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R2 Score: {r2}')

# PLOTTING MODEL COMPARISONS

In [ ]:
import matplotlib.pyplot as plt

# Define the attribute sets and their corresponding R2 scores
attr_sets = ['Attr Set 1', 'Attr Set 2', 'Attr Set 3', 'Attr Set 4']
r2_scores = [0.9534964916495487, 0.9011542128032501, 0.9359800615312223, 0.8927537138619698]

# Create a figure and axis
plt.figure(figsize=(10, 6))

# Plot the R2 scores for different attribute sets
plt.plot(attr_sets, r2_scores, marker='o', linestyle='-', color='blue', label='R2 Score')

# Annotate points with their R2 score values
for i, score in enumerate(r2_scores):
    plt.text(attr_sets[i], score, f'{score:.4f}', ha='center', va='bottom')

# Title and labels
plt.title('R2 Scores for Different Attribute Sets Using Random Forest')
plt.ylabel('R2 Score')
plt.xlabel('Attribute Sets')
plt.ylim(0, 1)  # Set y-axis limit for better visualization
plt.legend()
plt.grid(True)

# Save the figure
plt.savefig('/kaggle/working/RF_comparisons.png')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Define the attribute sets and their corresponding R2 scores
attr_sets = ['Attr Set 1', 'Attr Set 2', 'Attr Set 3', 'Attr Set 4']
r2_scores = [0.9534964916495487, 0.9011542128032501, 0.9359800615312223, 0.8927537138619698]

# Define hatch patterns for each bar
patterns = ['/', '\\', '|', '-']

# Create a smaller figure and axis
plt.figure(figsize=(10, 6))

# Plot the R2 scores as a bar chart with different patterns
bars = plt.bar(attr_sets, r2_scores, hatch=patterns[0], color='white', edgecolor='black')

# Apply different hatch patterns to each bar
for bar, pattern in zip(bars, patterns):
    bar.set_hatch(pattern)

# Annotate bars with their R2 score values (smaller font and close to bars)
for i, score in enumerate(r2_scores):
    plt.text(i, score + 0.005, f'{score:.4f}', ha='center', va='bottom', fontsize=8)

# Title and labels (smaller font size)

plt.ylabel('R2 Score', fontsize=9)
plt.xlabel('Attribute Sets', fontsize=9)
plt.ylim(0.8, 1)  # Set y-axis limit to focus on the range 0.8 to 1
plt.grid(False)

# Save the figure
plt.savefig('/kaggle/working/RF_comparisons_bar.png')

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Results from different models
models = ['Random Forest', 'XGBoost', 'SVR', 'CatBoost']
r2_scores = [0.9550701929644764, 0.9436145383374764, 0.6003614289271699, 0.1771460132644208]

# Create the line plot for R2 scores
plt.figure(figsize=(10, 6))

# Plot each model's R2 score with individual lines
plt.plot(models, r2_scores, marker='o', linestyle='-', color='b', label='R2 Score')

# Highlight individual scores with markers
for i, score in enumerate(r2_scores):
    plt.text(models[i], score, f'{score:.2f}', ha='center', va='bottom')

plt.title('R2 Scores of Different Models')
plt.ylabel('R2 Score')
plt.ylim(0, 1)
plt.xlabel('Models')
plt.grid(axis='y')
plt.legend()

# Display the plot
plt.savefig('/kaggle/working/r2_scores.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Metrics
metrics = ['MAE', 'MSE', 'RMSE', 'R2 Score']

# Results for each model
results = {
    'Random Forest': [0.203, 0.107, 0.327, 0.955],
    'XGBoost': [0.225, 0.134, 0.366, 0.944],
    'SVR': [0.531, 0.952, 0.976, 0.600],
    'CatBoost': [0.932, 1.960, 1.400, 0.177]
}

# Create a figure and axis
plt.figure(figsize=(12, 8))

# Plot each model's metrics
for model, values in results.items():
    plt.plot(metrics, values, marker='o', linestyle='-', label=model)

# Annotate points with their values
for model, values in results.items():
    for i, value in enumerate(values):
        plt.text(metrics[i], value, f'{value:.3f}', ha='center', va='bottom')

# Title and labels
plt.title('Model Performance Across Different Metrics')
plt.ylabel('Metric Value')
plt.xlabel('Metrics')
plt.legend()
plt.grid(True)

# Display the plot
plt.savefig('/kaggle/working/model_performances.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Metrics
metrics = ['MAE', 'MSE', 'RMSE', 'R2 Score']

# Results for each model
results = {
    'Random Forest': [0.203, 0.107, 0.327, 0.955],
    'XGBoost': [0.225, 0.134, 0.366, 0.944],
    'SVR': [0.531, 0.952, 0.976, 0.600],
    'CatBoost': [0.932, 1.960, 1.400, 0.177]
}

# Define the number of models and metrics
n_models = len(results)
n_metrics = len(metrics)

# Set up bar width and index positions for each group of bars
bar_width = 0.2
index = np.arange(n_metrics)

# Create a figure and axis
plt.figure(figsize=(12, 8))

# Plot each model's metrics as bars
for i, (model, values) in enumerate(results.items()):
    plt.bar(index + i * bar_width, values, bar_width, label=model)

# Annotate bars with their values
for i, (model, values) in enumerate(results.items()):
    for j, value in enumerate(values):
        plt.text(index[j] + i * bar_width, value + 0.02, f'{value:.3f}', ha='center', va='bottom')

# Title and labels
plt.title('Model Performance Across Different Metrics')
plt.ylabel('Metric Value')
plt.xlabel('Metrics')
plt.xticks(index + bar_width * (n_models - 1) / 2, metrics)
plt.legend()
plt.grid(True, axis='y')

# Display the plot
plt.savefig('/kaggle/working/model_performances_bar.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Results for each model
results = {
    'Random Forest': 0.203,
    'XGBoost': 0.225,
    'SVR': 0.531,
    'CatBoost': 0.932
}

# Define consistent hatch patterns for each model
patterns = ['/', '..', 'x', '--']

# Create a bar plot for MAE
plt.figure(figsize=(8, 5), facecolor='white')  # Set figure background to white
bars = plt.bar(results.keys(), results.values(), edgecolor='black', hatch=patterns, color='none')

# Annotate bars with their values
for bar, value in zip(bars, results.values()):
    plt.text(bar.get_x() + bar.get_width() / 2, value + 0.02, f'{value:.3f}', ha='center', va='bottom')

# Labels
plt.ylabel('MAE Value')
plt.xlabel('Models')
plt.ylim(0, 1)  # Set y-axis limit for better visualization

# Remove grid lines
plt.grid(False)

# Save and display the plot
plt.savefig('/kaggle/working/MAE_models_bar.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Results for each model
results = {
    'Random Forest': 0.107,
    'XGBoost': 0.134,
    'SVR': 0.952,
    'CatBoost': 1.960
}

# Define consistent hatch patterns for each model
patterns = ['/', '..', 'x', '--']

# Create a bar plot for MSE
plt.figure(figsize=(8, 5), facecolor='white')  # Set figure background to white
bars = plt.bar(results.keys(), results.values(), edgecolor='black', hatch=patterns, color='none')

# Annotate bars with their values
for bar, value in zip(bars, results.values()):
    plt.text(bar.get_x() + bar.get_width() / 2, value + 0.05, f'{value:.3f}', ha='center', va='bottom')

# Title and labels

plt.ylabel('MSE Value')
plt.xlabel('Models')
plt.ylim(0, 2.5)  # Set y-axis limit for better visualization
plt.grid(False)

# Save and display the plot
plt.savefig('/kaggle/working/MSE_models_bar.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Results for each model
results = {
    'Random Forest': 0.327,
    'XGBoost': 0.366,
    'SVR': 0.976,
    'CatBoost': 1.400
}

# Define consistent hatch patterns for each model
patterns = ['/', '..', 'x', '--']

# Create a bar plot for RMSE
plt.figure(figsize=(8, 5), facecolor='white')  # Set figure background to white
bars = plt.bar(results.keys(), results.values(), edgecolor='black', hatch=patterns, color='none')

# Annotate bars with their values
for bar, value in zip(bars, results.values()):
    plt.text(bar.get_x() + bar.get_width() / 2, value + 0.05, f'{value:.3f}', ha='center', va='bottom')

# Title and labels

plt.ylabel('RMSE Value')
plt.xlabel('Models')
plt.ylim(0, 1.6)  # Set y-axis limit for better visualization
plt.grid(False)

# Save and display the plot
plt.savefig('/kaggle/working/RMSE_models_bar.png')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Results for each model
results = {
    'Random Forest': 0.955,
    'XGBoost': 0.944,
    'SVR': 0.600,
    'CatBoost': 0.177
}

# Define consistent hatch patterns for each model
patterns = ['/', '..', 'x', '--']

# Create a bar plot for R² Score
plt.figure(figsize=(8, 5), facecolor='white')  # Set figure background to white
bars = plt.bar(results.keys(), results.values(), edgecolor='black', hatch=patterns, color='none')

# Annotate bars with their values
for bar, value in zip(bars, results.values()):
    plt.text(bar.get_x() + bar.get_width() / 2, value + 0.02, f'{value:.3f}', ha='center', va='bottom')

# Title and labels

plt.ylabel('R² Score Value')
plt.xlabel('Models')
plt.ylim(0, 1.1)  # Set y-axis limit for better visualization
plt.grid(False)

# Save and display the plot
plt.savefig('/kaggle/working/R2_models_bar.png')
plt.show()


In [ ]:
import pandas as pd
from IPython.display import display, Markdown

# Define the data
data = {
    'Model': ['Random Forest', 'XGBoost', 'SVR', 'CatBoost'],
    'Mean Absolute Error': [0.2031, 0.2254, 0.5311, 0.9317],
    'Mean Squared Error': [0.1070, 0.1343, 0.9520, 1.9602],
    'Root Mean Squared Error': [0.3272, 0.3665, 0.9757, 1.4001],
    'R2 Score': [0.9551, 0.9436, 0.6004, 0.1771]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Display the table in a readable format using markdown
table_md = df.to_markdown(index=False)
display(Markdown(table_md))

# Save the DataFrame to a CSV file
# df.to_csv('/kaggle/working/model_performance_table.csv', index=False)
